In [1]:
#ignore this file for now -- used for only testing and is very rough
import os
import json
from glob import glob

import numpy as np
import prody as pdy
import pandas as pd

import torch

In [2]:
jsons = glob('/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/*.json')[:100]
jsons

['/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/4X7F_C.json',
 '/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/1V2B_B.json',
 '/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/4C5S_B.json',
 '/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/3W15_B.json',
 '/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/2Q5C_A.json',
 '/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/5ZBR_B.json',
 '/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/6L8A_F.json',
 '/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/5J2S_A.json',
 '/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/2GJ4_A.json',
 '/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/4LZX_B.json',
 '/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/5YF4_A.json',
 '/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/1O75_B.json',
 '/home/ajindal/ssh_mnt/residue-packing/out_redo_w_rot-cat_v3/4DJT_A.json',
 '/home/ajin

In [3]:
def read_json(json_file):
    with open(json_file, 'r') as rf:
        return json.load(rf)

In [4]:
def get_pdb(json_file):
    pdb_name = os.path.basename(json_file).split('.')[0]
    return os.path.abspath(f'/home/ajindal/ssh_mnt/CLEAN_PDB_03062021/{pdb_name}.pdb')

In [5]:
GRAPH_ID, ONE_HOT, PHI, PSI, CHI, EDGE_NUM, NODE_NUM, CA_COORD, C_COORD, N_COORD, TARGET_CHI, EDGES = ([] for i in range(12))
RES_TYPES = np.array(['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL'], dtype='str')

MAX_EDGES = 500
MAX_NODES = 500

ROI_CAT = [59, 60]
ROI_NAME = 'TYR'

cata_1 = 0
cata_2 = 0

for json_file in jsons:
    data = read_json(json_file)
    df = pd.DataFrame.from_dict(data)
    
    resis_of_interest = df[df['res_name'] == ROI_NAME]
    roi_indices = resis_of_interest.index.to_list()
    roi_resnum = resis_of_interest['res_num'].values
    roi_cat = resis_of_interest['rota_category'].values
    roi_dmat = resis_of_interest['res_d_mat'].values
    
    viable_node_flag = False
    max_neigh_size = 4 # we want at 3 residues of interest (roi) which are all part of layer_1
    for idx, arr in enumerate([np.array(i)[:, roi_indices] < 15.0 for i in roi_dmat]):
        if np.sum(arr) > max_neigh_size:
            max_neigh_size = np.sum(arr)
            res_num_central_node = roi_resnum[idx]
            res_cat_central_node = roi_cat[idx]
            viable_node_flag = True
            
    if viable_node_flag:
        protein_ca = pdy.parsePDB(get_pdb(json_file)).select('name CA')
        roi_layer_1 = protein_ca.select(f'resname {ROI_NAME} exwithin 15.0 of resnum {res_num_central_node}').getResnums()[:4]
        roi_catas = [df[df['res_num'] == i]['rota_category'].values[0] for i in roi_layer_1]
        TARGET_CHI.append([res_cat_central_node] + roi_catas)
        g_rnames = [str(i) for i in list([res_num_central_node] + roi_layer_1)]
        GRAPH_ID.append(os.path.basename(json_file).split('.')[0] + '_' + '-'.join(g_rnames))
        all_phi = df['phi'].values
        all_psi = df['psi'].values
        all_ca_xyz = df['ca_xyz'].values
        all_c_xyz = df['c_xyz'].values
        all_n_xyz = df['n_xyz'].values
        all_o_xyz = df['o_xyz'].values
        all_res_name = df['res_name'].values
        all_chis = df['chis'].values
        all_res_cat = df['rota_category'].values
        
        central = df[(df['res_num'] == int(res_num_central_node))]
        d_mat = np.array(central['res_d_mat'])
        d_mat = np.array(d_mat[0][0])
        layer_1_idx = [neighbor for neighbor in list(np.where(d_mat < 15.0)[0]) if neighbor != central.index[0]]
        obfuscate = [central.index[0]]
        edge = []
        for idx in layer_1_idx:
            edge.append([central.index[0], idx, 0])
        
        for res_num_roi in roi_layer_1:
            layer_2_central = df[(df['res_num'] == int(res_num_roi))]
            d_mat_layer_2 = np.array(layer_2_central['res_d_mat'])
            d_mat_layer_2 = np.array(d_mat_layer_2[0][0])
            layer_2_idx = [neighbor for neighbor in list(np.where(d_mat_layer_2 < 15.0)[0]) if neighbor != layer_2_central.index[0]]
            obfuscate.append(layer_2_central.index[0])
            for idx in layer_2_idx:
                edge.append([layer_2_central.index[0], idx, 0])
                
        list_node = []
        for e in edge:
            list_node.append(e[0])
            list_node.append(e[1])
        list_node = list(set(list_node))
        neighbor_dict = dict(zip(list_node, [i for i in range(len(list_node))]))
        re_indexed_edge = []
        for e in edge:
            e[0] = neighbor_dict[e[0]]
            e[1] = neighbor_dict[e[1]]
        num_edge = len(edge)
        num_nodes = len(list_node)
        
        norm_edge = np.zeros((MAX_EDGES, 3))
        for i, v in enumerate(edge):
            norm_edge[i] = v
        EDGES.append([norm_edge])
        EDGE_NUM.append(num_edge)
        NODE_NUM.append(num_nodes)
        
        
        # ensuring that all node features have same dimension across all neighborhoods
        norm_x = np.zeros((MAX_NODES, 3))
        norm_xc = np.zeros((MAX_NODES, 3))
        norm_xn = np.zeros((MAX_NODES, 3))
        norm_phi = np.zeros((MAX_NODES, 1))
        norm_psi = np.zeros((MAX_NODES, 1))
        norm_resname = np.zeros((MAX_NODES, 20), dtype=bool)
        norm_chis = np.zeros((MAX_NODES, 1))

        for idx, i in enumerate(list_node):
            i = i.astype(int)
            norm_x[idx] = [all_ca_xyz[i][0], all_ca_xyz[i][1], all_ca_xyz[i][2]]
            norm_xc[idx] = [all_ca_xyz[i][0] - all_c_xyz[i][0], all_ca_xyz[i][1] - all_c_xyz[i][1], all_ca_xyz[i][2] - all_c_xyz[i][2]]
            norm_xn[idx] = [all_ca_xyz[i][0] - all_n_xyz[i][0], all_ca_xyz[i][1] - all_n_xyz[i][1], all_ca_xyz[i][1] - all_n_xyz[i][2]]
            norm_phi[idx] = all_phi[i]
            norm_psi[idx] = all_psi[i]
            t = RES_TYPES == all_res_name[i]
            norm_resname[idx] = list(t)
            norm_chis[idx] = int(all_res_cat[i])
            if i in obfuscate:
                norm_chis[idx] = 0

        PHI.append(norm_phi)
        PSI.append(norm_psi)
        ONE_HOT.append(norm_resname)
        CA_COORD.append(norm_x)
        C_COORD.append(norm_xc)
        N_COORD.append(norm_xn)
        CHI.append(norm_chis)

@> 857 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1168 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 4958 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 3456 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 6532 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> 3130 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 2325 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1775 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 2369 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 2701 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1392 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> 1930 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 2834 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 4115 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 2172 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1974 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1537 atoms and 1 coordinate set(s) wer

In [6]:
data_d = {
    'res_id' : GRAPH_ID,
    'num_node' : NODE_NUM,
    'num_edge' : EDGE_NUM,
    'target' : TARGET_CHI,
    'x' : CA_COORD,
    'x_c' : C_COORD,
    'x_n' : N_COORD,
    'one_hot' : ONE_HOT,
    'phi' : PHI,
    'psi' : PSI,
    'edge' : EDGES
}

In [7]:
data_dunbrack = {}

data_dunbrack["train"] = {}
data_dunbrack["valid"] = {}
data_dunbrack["test"] = {}

# edit it here
split_train_valid = 30
split_valid_test = 36

for key in data_d.keys():
    data_dunbrack["train"][key] = data_d[key][0:split_train_valid]
    data_dunbrack["valid"][key] = data_d[key][split_train_valid:split_valid_test]
    data_dunbrack["test"][key] = data_d[key][split_valid_test:]

In [8]:
torch.save(data_dunbrack, './small_test.pt')

In [9]:
data = torch.load('./small_test.pt')

In [10]:
def print_ith_item(i):
    for key in data['train'].keys():
        print(key + ": " + str(data['test'][key][i]))

In [11]:
print_ith_item(1)

res_id: 3VU9_B_347-375-379-380
num_node: 77
num_edge: 216
target: [60, 59, 60, 59, 59]
x: [[38.527  6.776 17.128]
 [38.444  4.298 20.098]
 [41.548  2.304 21.213]
 ...
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]]
x_c: [[-0.275  0.577 -1.429]
 [-1.325  0.751  0.068]
 [ 0.047  1.196  0.991]
 ...
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]]
x_n: [[  0.062  -1.441 -10.389]
 [  0.189  -0.715 -14.54 ]
 [  1.338  -0.716 -18.89 ]
 ...
 [  0.      0.      0.   ]
 [  0.      0.      0.   ]
 [  0.      0.      0.   ]]
one_hot: [[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
phi: [[ -81.7]
 [ -82.3]
 [ -59.7]
 [-146. ]
 [-125.3]
 [ -96. ]
 [-111. ]
 [ -75.8]
 [  31.7]
 [-119.7]
 [  83.4]
 [ -59.5]
 [ -62.8]
 [ -72.9]
 [ -57.6]
 [ -

In [12]:
len(data_d['res_id'])

42